In [8]:
import nltk
from sklearn.datasets import fetch_20newsgroups
import pandas as pd
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kang9\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [16]:
#뉴스 다운로드 및 전처리
def get_news() :
    #20newsgroup 다운로드
    dataset=fetch_20newsgroups(shuffle=True, random_state=1,remove=('headers','footers','quotes'))
    documents=dataset.data
    
    news_df=pd.DataFrame({'document':documents})
    
    #전처리
    news_df['clean_doc']=news_df['document'].str.replace("[^a-zA-Z]"," ") #특수문자 제거
    news_df['clean_doc']=news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
    news_df['clean_doc']=news_df['clean_doc'].apply(lambda x: x.lower())#전체 단어에 대한 소문자 변환
    
    tokenized_doc=news_df['clean_doc'].apply(lambda x: x.split()) #토큰화
    
    stop_words=stopwords.words('english') #NLTK불용어 조회
    return tokenized_doc.apply(lambda x: ' '.join([item for item in x if item not in stop_words]))

In [17]:
tokenized_docs=get_news()

In [18]:
tokenized_docs

0        well sure story seem biased disagree statement...
1        yeah expect people read actually accept hard a...
2        although realize principle strongest points wo...
3        notwithstanding legitimate fuss proposal much ...
4        well change scoring playoff pool unfortunately...
                               ...                        
11309    danny rubenstein israeli journalist speaking t...
11310                                                     
11311    agree home runs clemens always memorable kinda...
11312    used deskjet orange micros grappler system upd...
11313    argument murphy scared hell came last year han...
Name: clean_doc, Length: 11314, dtype: object

In [19]:
#공백으로 토큰 분리
def my_tokenizer(text):
    return text.split()

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

tfidf_vect=TfidfVectorizer(tokenizer=my_tokenizer)
tfidf=tfidf_vect.fit_transform(tokenized_docs)

#모델선언
lda=LatentDirichletAllocation(n_components=20)
lda_output=lda.fit_transform(tfidf)

In [21]:
!pip install pyLDAvis

  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97717 sha256=68a35a1fadc470c621c66aec097e7be38f477b00c505fcbb7250ed5055d8bcc4
  Stored in directory: c:\users\kang9\appdata\local\pip\cache\wheels\57\de\11\0a038be70c2c212ce45fa0f4f9da165bb5dd87de1288394dc3
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491062 sha256=171fff7b8d6b4242aa26a4e7390b4b1631042a58b799e54427b44a5bb5b9cca1
  Stored in directory: c:\users\kang9\appdata\local\pip\cache\wheels\6e\9c\ed\4499c9865ac1002697793e0ae05ba6be33553d098f3347fb94
  Created wheel for funcy: filename=funcy-1.14-py2.py3-none-any.whl size=32047 sha256=db5f11026897e58afbb08597cabfe4b01a61c62b75547b20e7e0ed6d077568ee
  Stored in directory: c:\users\kang9\appdata\local\pip\cache\wheels\5e\e0\ba\0dd7bb3f79264f8f60690da62918081bb8c3fb7442c38bfddd
Successfully built pyLDAvis future funcy


In [22]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
vis = pyLDAvis.sklearn.prepare(lda, tfidf, tfidf_vect, mds='tsne')
pyLDAvis.display(vis)